# Classification with stumbleupon data

Project 4 has been changed since scraping was untenable. The project now focuses on the stumbleupon kaggle dataset. For more information on this dataset, [check out the website here](https://www.kaggle.com/c/stumbleupon).

---

## 1. Load in the dataset

This is the only part completed for you.

---

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('white')

%matplotlib inline

In [8]:
su = pd.read_csv('../dataset/evergreen.tsv', delimiter='\t')

## 2. Clean up/examine your data

Some of the columns may have values that need changing or that are of the wrong type. There could also be columns that aren't very useful.

---

In [9]:
su['embed_ratio'] = su['embed_ratio'].map(lambda x: 0 if x < 0 else x)
#su.embed_ratio

In [10]:
su.image_ratio = su.image_ratio.map(lambda x: 0 if x == -1 else x)
#su.image_ratio

In [11]:
for i in su.columns:
    if su[i].dtype == object:
        print i

url
boilerplate
alchemy_category
alchemy_category_score
is_news
news_front_page


In [12]:
su['embed_ratio'].min()

0.0

In [13]:
su['is_news'] = su['is_news'].map(lambda x: 0 if x == '?' else x)
su['news_front_page'] = su['news_front_page'].map(lambda x: 0 if x == '?' else x)
su['alchemy_category_score'] = su['alchemy_category_score'].map(lambda x: 0 if x == '?' else x)
su['alchemy_category_score'] = su['alchemy_category_score'].map(lambda x: float(x))
su['url'] = su['url'].map(lambda x: '' if x == '?' else x)
su['boilerplate'] = su['boilerplate'].map(lambda x: '' if x == '?' else x)
su['alchemy_category'] = su['alchemy_category'].map(lambda x: 'unknown' if x == '?' else x)
su['news_front_page'] = su['news_front_page'].map(lambda x: int(x) if type(x)== str else x)
su['is_news'] = su['is_news'].map(lambda x: int(x) if type(x)== str else x)


#su['embed_ratio'] = su['embed_ratio'].map(lambda x: 0 if x < 0 else x)

## 3. Use statsmodels' logistic regression function to look at variable significance

The **`import statsmodels.formula.api as smf`** code below gives us access to a statsmodels api that can run logistic regressions using patsy-style formulas.

Ex:

```python
formula = 'target ~ var1 + var2 + C(var3) -1'
logreg = smf.logit(formula, data=data)
logreg_results = logreg.fit()
print logreg_results.summary()
```

---

In [14]:
import statsmodels.formula.api as smf

In [15]:
cat_vars = []

for index, coltype in enumerate(su.dtypes):
    if (coltype != np.float64) and (coltype != np.int64):
        cat_vars.append(su.columns[index])

cat_vars.append('hasDomainLink')
cat_vars.append('lengthyLinkDomain')
cat_vars.append('is_news')
cat_vars.append('news_front_page')

In [16]:
cont_vars = list((set(su.columns) - set(cat_vars)) - set(['framebased', 'label','urlid']))
cont_vars

['parametrizedLinkRatio',
 'alchemy_category_score',
 'commonlinkratio_3',
 'embed_ratio',
 'spelling_errors_ratio',
 'avglinksize',
 'commonlinkratio_1',
 'commonlinkratio_2',
 'linkwordscore',
 'commonlinkratio_4',
 'compression_ratio',
 'numwords_in_url',
 'frameTagRatio',
 'image_ratio',
 'non_markup_alphanum_characters',
 'numberOfLinks',
 'html_ratio']

In [17]:
cat_vars

['url',
 'boilerplate',
 'alchemy_category',
 'hasDomainLink',
 'lengthyLinkDomain',
 'is_news',
 'news_front_page']

In [18]:
su.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,1,1,24,0,5424,170,8,0.152941,0.079130,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,1,1,40,0,4973,187,9,0.181818,0.125448,1
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,1,1,55,0,2240,258,11,0.166667,0.057613,1
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,1,0,24,0,2737,120,5,0.041667,0.100858,1
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,1,1,14,0,12032,162,10,0.098765,0.082569,0


In [19]:
from sklearn.preprocessing import StandardScaler
norm_su = su


sc = StandardScaler()
sc.fit(su[cont_vars])
norm_su[cont_vars] = sc.transform(su[cont_vars])
#del norm_su['framebased']
norm_su.head()

# x_train_std =  sc.transform(x_train)
# x_test_std =  sc.transform(x_test)

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,1.137724,-0.081941,1.025209,-0.055867,-0.468925,-0.354329,...,1,1,-0.298017,0,-0.032969,-0.048784,0.940134,-0.108706,-0.278840,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.488719,0.106291,0.195899,0.509021,1.269555,1.309802,...,1,1,0.486615,0,-0.083787,0.045947,1.249454,0.048857,0.305801,1
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,1.763819,-0.043965,0.461723,0.733479,0.292718,-0.091244,...,1,1,1.222208,0,-0.391737,0.441592,1.868095,-0.033815,-0.550424,1
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,1.174304,-0.141396,-0.335913,-0.777467,-0.785603,-0.678320,...,1,0,-0.298017,0,-0.335736,-0.327407,0.012172,-0.715857,-0.004575,1
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.926483,-0.009903,0.156408,0.055491,0.327122,-0.083337,...,1,1,-0.788412,0,0.711608,-0.093364,1.558775,-0.404307,-0.235429,0


In [20]:
del norm_su['framebased']

### 3.1 Run a logistic regression predicting evergreen from the numeric columns

And print out the results as shown in the example above.

---

In [21]:
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.cross_validation import train_test_split
import patsy


# formula = 'label ~  parametrizedLinkRatio + alchemy_category_score + \
# linkwordscore + embed_ratio+ spelling_errors_ratio + commonlinkratio_1 + commonlinkratio_2 + commonlinkratio_3 \
# +commonlinkratio_4 + compression_ratio + avglinksize + numwords_in_url + image_ratio+frameTagRatio+numberOfLinks \
# + non_markup_alphanum_characters + html_ratio'

formula = 'label ~ spelling_errors_ratio + linkwordscore + non_markup_alphanum_characters + frameTagRatio -1'

y_mat, x_mat = patsy.dmatrices(formula, data = su)
# X = su[cont_vars]
# Y = su['label']

x_train, x_test, y_train, y_test = train_test_split(x_mat,y_mat, test_size=0.3)




In [22]:


logreg = LogisticRegression()
logreg = logreg.fit(x_train, y_train)
lorreg = logreg.predict(x_test)

logreg.score(x_test,y_test)

/home/dsi-student/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6002703920684993

### 3.2 Run a logistic regression predicting evergreen from the numeric columns and a categorical variable of alchemy_category

And print out the results as shown in the example.

---

In [23]:
formula = 'label ~ C(alchemy_category) + spelling_errors_ratio + linkwordscore + non_markup_alphanum_characters + frameTagRatio -1'

y_mat, x_mat = patsy.dmatrices(formula, data = norm_su)

x_train, x_test, y_train, y_test = train_test_split(x_mat,y_mat, test_size=0.3)


In [24]:
logreg = LogisticRegression()
logreg = logreg.fit(x_train, y_train)
lorreg = logreg.predict(x_test)

logreg.score(x_test,y_test)

/home/dsi-student/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6502929247408743

## 4. Use sklearn to cross-validate the accuracy of the model above

Normalize the numeric and categorical columns of the predictor matrix.

---

In [25]:
formula = 'label ~ C(alchemy_category) + spelling_errors_ratio + linkwordscore + non_markup_alphanum_characters + frameTagRatio -1'

y_mat, x_mat = patsy.dmatrices(formula, data = norm_su)

x_train, x_test, y_train, y_test = train_test_split(x_mat,y_mat, test_size=0.3)


In [26]:
logreg = LogisticRegression()
logreg = logreg.fit(x_train, y_train)
lorreg = logreg.predict(x_test)

logreg.score(x_test,y_test)

/home/dsi-student/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.66381252816584047

## 5. Gridsearch regularization parameters for logistic regression

Find the best regularization type (Ridge, Lasso) across a set of regularization strengths.

[NOTE: C is the inverse of the regularization strength. Lower C values are stronger regularization. Having a C higher than 1 will significantly slow down the search. I'm not particularly interested in values over 1, since this is the default regularization strength in LogisticRegression.]

**After you find the best set of parameters, build a Logistic Regression with those parameters and crossvalidate the score.**

[NOTE 2: to run Lasso regularization the solver should be `'liblinear'`]

---

In [27]:
from sklearn.grid_search import GridSearchCV
formula = 'label ~ C(news_front_page) + spelling_errors_ratio + linkwordscore + non_markup_alphanum_characters + frameTagRatio -1'

y_mat, x_mat = patsy.dmatrices(formula, data = norm_su)

y_mat = np.ravel(y_mat)

x_train, x_test, y_train, y_test = train_test_split(x_mat,y_mat, test_size=0.3)

In [28]:
log = LogisticRegression()
Cs = np.logspace(0.0, 5.0, 20)

search_parameters = {
    "penalty":             ['l1','l2'], 
    "C":                   Cs,  
    "fit_intercept":       [False, True], 
    "class_weight":        [None, "balanced"], 
    "intercept_scaling":   [2, 1],         
    "solver":              ['liblinear'],
    "warm_start":          [False, True]
}

grid_log = GridSearchCV(log, search_parameters)

In [29]:
grid_log.fit(x_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'warm_start': [False, True], 'C': array([  1.00000e+00,   1.83298e+00,   3.35982e+00,   6.15848e+00,
         1.12884e+01,   2.06914e+01,   3.79269e+01,   6.95193e+01,
         1.27427e+02,   2.33572e+02,   4.28133e+02,   7.84760e+02,
         1.43845e+03,   2.63665e+03,   4.83293e+03,  ...near'], 'fit_intercept': [False, True], 'penalty': ['l1', 'l2'], 'class_weight': [None, 'balanced']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [30]:
grid_log.predict(x_test)

array([ 0.,  0.,  0., ...,  0.,  1.,  0.])

In [31]:
np.mean(y_test)

array(0.5056331680937359)

In [32]:
print "Best Params:", grid_log.best_params_
print "Best Score:", grid_log.best_score_

Best Params: {'warm_start': False, 'C': 1.0, 'intercept_scaling': 2, 'fit_intercept': False, 'solver': 'liblinear', 'penalty': 'l1', 'class_weight': 'balanced'}
Best Score: 0.603361669243


## 6. Gridsearch neighbors for kNN

Find the best number of neighbors with your predictors to predict the `label` target variable.

Start by bulding a kNN model with a set number of neighbors, then use gridsearch to run through a series of neighbors.

---

In [33]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV
formula = 'label ~ C(alchemy_category) + spelling_errors_ratio + linkwordscore + non_markup_alphanum_characters + frameTagRatio -1 '

y_mat, x_mat = patsy.dmatrices(formula, data = norm_su)

y_mat = np.ravel(y_mat)

x_train, x_test, y_train, y_test = train_test_split(x_mat,y_mat, test_size=0.3)




In [34]:
knn = KNeighborsClassifier()

search_parameters = {
    'n_neighbors':  [1,50], 
    'weights':      ("uniform", "distance"),
    'algorithm':    ("ball_tree", "kd_tree", "brute", "auto"),
    'p':            [1,2]
}


grid_knn = GridSearchCV(knn, search_parameters)

In [35]:
grid_knn.fit(x_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [1, 50], 'weights': ('uniform', 'distance'), 'algorithm': ('ball_tree', 'kd_tree', 'brute', 'auto'), 'p': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [36]:
np.mean(y_test)

array(0.5051825146462371)

In [37]:
grid_knn.predict(x_test)

array([ 0.,  0.,  1., ...,  1.,  1.,  0.])

In [38]:
print "Best Params:", grid_log.best_params_
print "Best Score:", grid_log.best_score_

Best Params: {'warm_start': False, 'C': 1.0, 'intercept_scaling': 2, 'fit_intercept': False, 'solver': 'liblinear', 'penalty': 'l1', 'class_weight': 'balanced'}
Best Score: 0.603361669243


## 7. Choose a new target from alchemy_category to predict with logistic regression

**Ideally your category choice will have a small fraction of the total rows, but not TOO small!**

---

### 7.1 Chose your target category, create the Y vector, and check the fraction of instances

---

In [39]:
cat_su = norm_su
cat_su['is_sports'] = cat_su.alchemy_category.map(lambda x: 1 if x == 'sports' else 0)
cat_su.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,is_sports
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,1.137724,-0.081941,1.025209,-0.055867,-0.468925,-0.354329,...,1,-0.298017,0,-0.032969,-0.048784,0.940134,-0.108706,-0.278840,0,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.488719,0.106291,0.195899,0.509021,1.269555,1.309802,...,1,0.486615,0,-0.083787,0.045947,1.249454,0.048857,0.305801,1,0
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,1.763819,-0.043965,0.461723,0.733479,0.292718,-0.091244,...,1,1.222208,0,-0.391737,0.441592,1.868095,-0.033815,-0.550424,1,0
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,1.174304,-0.141396,-0.335913,-0.777467,-0.785603,-0.678320,...,0,-0.298017,0,-0.335736,-0.327407,0.012172,-0.715857,-0.004575,1,0
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.926483,-0.009903,0.156408,0.055491,0.327122,-0.083337,...,1,-0.788412,0,0.711608,-0.093364,1.558775,-0.404307,-0.235429,0,1


### 7.2 Use patsy to create an X matrix of the numeric predictors and all two-way interactions between them

Ex:

```python
import patsy

formula_interactions = '~ (var1 + var2 + var3)**2 -1'
X_interactions = patsy.dmatrix(formula_interactions, data=data
```

Get the column names from the `design_info` property of the patsy X matrix.

---

In [40]:
formula_interactions = 'is_sports ~ (spelling_errors_ratio + linkwordscore + non_markup_alphanum_characters + frameTagRatio)**2 -1'
y_mat, x_mat = patsy.dmatrices(formula_interactions, data=cat_su)

In [41]:
y_mat = y_mat.ravel()

In [42]:
x_mat.design_info

DesignInfo(['spelling_errors_ratio',
            'linkwordscore',
            'non_markup_alphanum_characters',
            'frameTagRatio',
            'spelling_errors_ratio:linkwordscore',
            'spelling_errors_ratio:non_markup_alphanum_characters',
            'spelling_errors_ratio:frameTagRatio',
            'linkwordscore:non_markup_alphanum_characters',
            'linkwordscore:frameTagRatio',
            'non_markup_alphanum_characters:frameTagRatio'],
           factor_infos={EvalFactor('spelling_errors_ratio'): FactorInfo(factor=EvalFactor('spelling_errors_ratio'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1),
                         EvalFactor('linkwordscore'): FactorInfo(factor=EvalFactor('linkwordscore'),
                                    type='numerical',
                                    state=<factor state>,
                               

### 7.3 Normalize the predictor matrix columns

---

In [43]:
#already done above

### 7.4 Gridsearch a logistic regression to predict accuracy on your new target from the interaction predictors

Include Ridge and Lasso.

---

In [44]:
new_log = LogisticRegression()
Cs = np.logspace(0.0, 5.0, 20)

search_parameters = {
    "penalty":             ['l1','l2'], 
    "C":                   Cs,  
    "fit_intercept":       [False, True], 
    "class_weight":        [None, "balanced"], 
    "intercept_scaling":   [2, 1],         
    "solver":              ['liblinear'],
    "warm_start":          [False, True]
}

grid_newlog = GridSearchCV(log, search_parameters)

In [45]:
grid_newlog.fit(x_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'warm_start': [False, True], 'C': array([  1.00000e+00,   1.83298e+00,   3.35982e+00,   6.15848e+00,
         1.12884e+01,   2.06914e+01,   3.79269e+01,   6.95193e+01,
         1.27427e+02,   2.33572e+02,   4.28133e+02,   7.84760e+02,
         1.43845e+03,   2.63665e+03,   4.83293e+03,  ...near'], 'fit_intercept': [False, True], 'penalty': ['l1', 'l2'], 'class_weight': [None, 'balanced']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [46]:
grid_newlog.predict(x_test)

array([ 0.,  0.,  1., ...,  1.,  1.,  0.])

In [47]:
from sklearn.cross_validation import cross_val_score, KFold

cv = KFold(len(y_test), 5, shuffle=True, random_state = 7)
scores = cross_val_score(new_log, x_test, y_test, cv=cv)
print scores, scores.mean()

[ 0.61936937  0.66666667  0.62837838  0.66666667  0.71106095] 0.658428405832


In [48]:
print "Best Params:", grid_log.best_params_
print "Best Score:", grid_log.best_score_

Best Params: {'warm_start': False, 'C': 1.0, 'intercept_scaling': 2, 'fit_intercept': False, 'solver': 'liblinear', 'penalty': 'l1', 'class_weight': 'balanced'}
Best Score: 0.603361669243


### 7.5 Build a logistic regression with the optimal parameters, and look at the coefficients

---

In [49]:
from sklearn.linear_model import Lasso
lasso =  Lasso(alpha=1.0, fit_intercept=False, warm_start=False)

In [50]:
lasso.fit(x_train,y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=False, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [51]:
lasso.predict(x_test)

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [52]:
lasso.score(x_test,y_test)

-1.0209471766848823

In [53]:
print lasso.coef_

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0.  0. -0.]


In [54]:
print x_test[0]

[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          1.          0.
 -0.79215951  0.87893122 -0.45247032  0.48885579]


In [55]:
print y_test[0]

0.0


### 7.6 Gridsearch parameters for a logistic regression with the same target and predictors, but score based on precision rather than accuracy

Look at the documentation.

---

In [56]:
from sklearn.metrics import precision_score


In [57]:
log = LogisticRegression()
Cs = np.logspace(0.0, 5.0, 20)

search_parameters = {
    "penalty":             ['l1','l2'], 
    "C":                   Cs,  
    "fit_intercept":       [False, True], 
    "class_weight":        [None, "balanced"], 
    "intercept_scaling":   [2, 1],         
    "solver":              ['liblinear'],
    "warm_start":          [False, True]
}

grid_finallog = GridSearchCV(log, search_parameters, scoring='precision')

In [58]:
grid_finallog.fit(x_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'warm_start': [False, True], 'C': array([  1.00000e+00,   1.83298e+00,   3.35982e+00,   6.15848e+00,
         1.12884e+01,   2.06914e+01,   3.79269e+01,   6.95193e+01,
         1.27427e+02,   2.33572e+02,   4.28133e+02,   7.84760e+02,
         1.43845e+03,   2.63665e+03,   4.83293e+03,  ...near'], 'fit_intercept': [False, True], 'penalty': ['l1', 'l2'], 'class_weight': [None, 'balanced']},
       pre_dispatch='2*n_jobs', refit=True, scoring='precision', verbose=0)

In [59]:
print "Best Params:", grid_finallog.best_params_
print "Best Score:", grid_finallog.best_score_

Best Params: {'warm_start': True, 'C': 1.0, 'intercept_scaling': 1, 'fit_intercept': True, 'solver': 'liblinear', 'penalty': 'l1', 'class_weight': 'balanced'}
Best Score: 0.659842465738


In [60]:
#precision favors RIDGE!!!

## [BONUS] 8. Build models predicting from words

This is a bit of the NLP we covered in the pipeline lecture!

---

### 8.1 Choose 'body' or 'title' from the boilerplate to be the basis of your word predictors

You will need to parse the json from the boilerplate field.

---

In [61]:
su['boilerplate'][0]

'{"title":"IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries","body":"A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees in its crystal ball The predictions are part of an annual tradition for the Armonk New York based company which surveys its 3 000 researchers to find five ideas expected to take root in the next five years IBM the world s largest provider of computer services looks to Silicon Valley for input gleaning many ideas from its Almaden research center in San Jose

### 8.2 Use CountVectorizer to create your predictor matrix from the string column

It is up to you what range of ngrams and features, and whether or not you want the columns binary or counts.

---

### 8.3 Gridsearch a logistic regression predicting accuracy of your chosen target category from word predictor matrix

---

### 8.4 Do the same as above, but score the gridsearch based on precision rather than accuracy

---

### 8.5 Build a logistic regression with optimal precision categories

Print out the top 20 or 25 word features as ranked by their coefficients.

---